# Problem 1 - Part 1

The taks: 
- read a list of integers from a file
- find how many entries are greater than the prevous entry

example:
```
199 (N/A - no previous measurement)
200 (increased)
208 (increased)
210 (increased)
200 (decreased)
```
Solution here is `3`

In [ ]:
list_inputs = []
with open('./data/aoc2021_day1_input.txt', 'r') as f:
    for line in f.readlines():
        list_inputs.append(int(line))

In [ ]:
counts = 0
for i, value in enumerate(list_inputs[:-1]):
    counts += 1 if list_inputs[i + 1] > value else 0
    
print(f"Final count is {counts}")

# Possible way to improove the solution?

- `int(line)` is prone to fail (example `int('7 a')`)
- use `zip` would avoid to indexing directly the list

In [ ]:
import re
with open('./data/aoc2021_day1_input.txt', 'r') as f:
    # re allows to find any value that fit a user defined regex
    list_inputs = [int(re.findall('\d+', line)[0]) for line in f.readlines()]

# list_inputs
# [1, 2, 3]
# list_inputs[:-1]
# [1, 2]
# list_inputs[1:]
# [2, 3]

counts = len([0 for value1, value2 in zip(list_inputs[:-1], list_inputs[1:]) if value2 > value1])
print(f"Final count is {counts}")

# Problem 1 - Part 2

Solve the same proble but averaging three measurement in a sliding window fashion
```
199  A       -> A: 607 (N/A - no previous sum)
200  A B     -> B: 618 (increased)
208  A B C   -> C: 618 (no change)
210    B C D -> D: 617 (decreased)
200  E   C D ...
```

In [ ]:
def count_depth_increase(list_inputs: list) -> int:
    return len([0 for value1, value2 in zip(list_inputs[:-1], list_inputs[1:]) if value2 > value1])

# same strategy
# list_inputs
# [1, 2, 3, 4, 5]
# Sliding window
# list_inputs[:-2]
# [1, 2, 3]
# list_inputs[1:-1]
# [2, 3, 4]
# list_inputs[2:]
# [3, 4, 5]

new_inputs = [(value1 + value2 + value3)/3 for value1, value2, value3 in zip(list_inputs[:-2],
                                                                         list_inputs[1:-1],
                                                                         list_inputs[2:])]
counts = count_depth_increase(new_inputs)
print(f"Final count is {counts}")

# Possible way to improove the solution?

* Sliding window size is fixed, what if we'll need to average 5 values? The code does not generalize

In [ ]:
import itertools

# list_inputs
# [1, 2, 3, 4, 5]
# itertools.islice(list_inputs, 3)
# [1, 2, 3]
# itertools.islice(list_inputs, 4)
# [1, 2, 3, 4]


def sliding_window_generator(list_inputs, window_size=3):
    for i in range(len(list_inputs)):
        # for each loop queue a sub list from i to -1
        queue_list = list_inputs[i:]
        
        # list() is required because islice returns a iterator without lenhts 
        window_slice = list(itertools.islice(queue_list, window_size))
        
        # check if window is larger that slice
        if len(window_slice) == window_size:
            yield sum(window_slice) / window_size
            
new_inputs = [sum_values for sum_values in sliding_window_generator(list_inputs, 10)]
counts = count_depth_increase(new_inputs)
print(f"Final count is {counts}")

# Problem 2 - Part 1

- read a series of instructions from a file
```
forward 5
down 5
forward 8
up 3
```
- starting from `horizontal position = 0` and `depth = 0` apply the instructions as follows:

    - forward X increases the horizontal position by X units.
    - down X increases the depth by X units.
    - up X decreases the depth by X units.


- after applying all instructions iteratively find the results by multiplying `horizontal position * depth`

In [ ]:
with open('./data/aoc2021_day2_input.txt', 'r') as f:
    list_direction_x = []
    for line in f.readlines():
        direction = ''.join(re.findall('[a-z]', line))
        x = int(re.findall('\d+', line)[0])
        list_direction_x.append([direction, x])

In [ ]:
position = [0, 0]
for direction, x in list_direction_x:
    if direction == 'forward':
        position[0] += x
    
    elif direction == 'up':
        position[1] -= x
    
    elif direction == 'down':
        position[1] += x
        
print(f'postion (h-pos, depth) = ({position[0]}, {position[1]}) - result {position[0] * position[1]}')

# Possible way to improove the solution?

- Saving the instructions/positions as list makes them a bit hard to use/read.

In [ ]:
from collections import namedtuple
# but dataclasses would also be great (probably better)

with open('./data/aoc2021_day2_input.txt', 'r') as f:
    list_instructions = []
    instruction = namedtuple('instruction', 'direction x')
    
    for line in f.readlines():
        direction = ''.join(re.findall('[a-z]', line))
        x = int(re.findall('\d+', line)[0])
        list_instructions.append(instruction(direction=direction, x=x))

In [ ]:
class SubmarinePosition:
    def __init__(self, h_pos: int, depth: int):
        self.h_pos = h_pos
        self.depth = depth
    
    def forward(self, x: int):
        self.h_pos += x
        
    def up(self, x: int):
        self.depth -= x
    
    def down(self, x: int):
        self.depth += x


In [ ]:
class SubmarinePosition:
    def __init__(self, h_pos: int, depth: int):
        self.h_pos = h_pos
        self.depth = depth
    
    def forward(self, x: int):
        self.h_pos += x
        
    def up(self, x: int):
        self.depth -= x
    
    def down(self, x: int):
        self.depth += x
        
    def apply_instruction(self, instruction: instruction):
        # find the right method to apply
        method = getattr(self, instruction.direction)
        
        # apply method
        method(instruction.x)
        
    def result(self):
        return self.h_pos * self.depth

In [ ]:
submarine = SubmarinePosition(h_pos=0, depth=0)

for instruction in list_instructions:
    submarine.apply_instruction(instruction)
    
print(f'postion (h-pos, depth) = ({submarine.h_pos}, {submarine.depth}) - result {submarine.result()}')

# Problem 2 - Part 2
- Your submarine now has three initial state parameters `horizontal position = 0`, `depth = 0` and `aim=0`
- The new rules are:
    - down X increases your aim by X units.
    - up X decreases your aim by X units.
    - forward X does two things:
       - It increases your horizontal position by X units.
       - It increases your depth by your aim multiplied by X. 

In [ ]:
class BetterSubmarinePosition:
    def __init__(self, h_pos: int, depth: int, aim: int):
        self.h_pos = h_pos
        self.depth = depth
        self.aim = aim # change
    
    def forward(self, x: int):
        self.h_pos += x
        self.depth += self.aim * x # change
        
    def up(self, x: int):
        self.aim -= x # change
    
    def down(self, x: int): # change
        self.aim += x
        
    def apply_instruction(self, instruction: instruction):
        # find the right method to apply
        method = getattr(self, instruction.direction)
        
        # apply method
        method(instruction.x)
        
    def result(self):
        return self.h_pos * self.depth

In [ ]:
submarine = BetterSubmarinePosition(h_pos=0, depth=0, aim=0)

for instruction in list_instructions:
    submarine.apply_instruction(instruction)
    
print(f'postion (h-pos, depth) = ({submarine.h_pos}, {submarine.depth}) - result {submarine.result()}')

# Possible way to improove the solution?

* a lot of copy paste. What if we still need the `standard` submarine and something in our model change? 

In [ ]:
class SubmarinePosition:
    def __init__(self, h_pos: int, depth: int):
        self.h_pos = h_pos
        self.depth = depth
    
    def forward(self, x: int):
        self.h_pos += x
        
    def up(self, x: int):
        self.depth -= x
    
    def down(self, x: int):
        self.depth += x
        
    def apply_instruction(self, instruction: instruction):
        # find the right method to apply
        method = getattr(self, instruction.direction)
        
        # apply method
        method(instruction.x)
        
    def result(self):
        return self.h_pos * self.depth

In [ ]:
from abc import ABC
class BetterSubmarinePosition(SubmarinePosition, ABC):
    def __init__(self, h_pos: int, depth: int, aim: int):
        super().__init__(h_pos=h_pos, depth=depth)
        self.aim = aim # change
    
    def forward(self, x: int):
        self.h_pos += x
        self.depth += self.aim * x # change
        
    def up(self, x: int):
        self.aim -= x # change
    
    def down(self, x: int): # change
        self.aim += x
        
    # self.result and self.apply_instruction are inerithed from SubmarinePosition

In [ ]:
submarine = BetterSubmarinePosition(h_pos=0, depth=0, aim=0)

for instruction in list_instructions:
    submarine.apply_instruction(instruction)
    
print(f'postion (h-pos, depth) = ({submarine.h_pos}, {submarine.depth}) - result {submarine.result()}')

# Problem 3 - Part 1 - Part 2

In [ ]:
import re
from typing import List, Set

class RowOrColumn:
    def __init__(self, entries: Set[int]):
        self.entries = entries
        self.numbers_matched = set()
        
    def test_number(self, num: int) -> bool:
        if num in self.entries:
            self.numbers_matched.add(num)
            return self._check_win()
        
        return False
        
    def _check_win(self) -> bool:
        if len(self.entries.difference(self.numbers_matched)) == 0:
            return True
        
        return False
    
    def __repr__(self):
        return f'{self.entries=}, {self.numbers_matched=}'
    
class Board:
    def __init__(self, idx_board: int):
        self.idx_board = idx_board
        self.rows = []
        self.columns = []
        
        self.extractions = set()
    
    def add_row(self, new_row: List[int]):
        new_row = set(new_row)
        new_row = RowOrColumn(entries=new_row)
        self.rows.append(new_row)
        
        
    def add_column(self, new_column: List[int]):
        new_column = set(new_column)
        new_column = RowOrColumn(entries=new_column)
        self.columns.append(new_column)
        
        
    def check_victory(self, num: int):
        self.extractions.add(num)
        for i, row in enumerate(self.rows):
            win = row.test_number(num)
            
            if win:
                return (self.idx_board, 'row', i, num, self.check_result(num))
        
        for i, col in enumerate(self.columns):
            win = col.test_number(num)
            
            if win:
                return (self.idx_board, 'col', i, num, self.check_result(num))
        
        return False
    
    def check_result(self, num: int) -> int:
        not_extracted = set()
        for i, row in enumerate(self.rows):
            not_extracted = not_extracted.union(row.entries.difference(row.numbers_matched))
            
        return sum(not_extracted) * num
            
    def __repr__(self):
        return f'Board {self.idx_board}'
    
with open('./data/aoc2021_day4_input.txt', 'r') as f:
    header = f.readline()
    extacted_numbers = [int(h) for h in header.split(',')]
    
    dict_boards, idx_board = {}, -1
    for i, line in enumerate(f.readlines()):
        if line == '\n':
            idx_board += 1
            new_board = Board(idx_board=idx_board)
            columns = [list() for i in range(5)]
            
        else:
            int_in_line = re.findall('\d+', line)
            # setup rows
            row = [int(entry) for entry in int_in_line]
            new_board.add_row(row)
            
            # columns 
            for ii, entry in enumerate(row):
                columns[ii].append(entry)
            
            if len(columns[0]) == 5:
                for c in columns:
                    new_board.add_column(c)
                    
                dict_boards[idx_board] = new_board
                

winners, winners_set = [], set()
for num in extacted_numbers:
    for board in dict_boards.values():
        winner = board.check_victory(num)
        
        if winner and winner[0] not in winners_set:
            winners_set.add(winner[0])
            winners.append(winner)
            
winners[0], winners[-1]